### Downloading Libraries

In [3]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 8.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=464212 sha256=94f17dd21a181feabb1245bf75c807d71aaf3f1fe28bd046173633496d308154
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


## Importing Libraries

In [55]:
import numpy as np
import pandas as pd
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import csr_matrix
from IPython.display import display
import pickle
import os

import warnings
warnings.filterwarnings('ignore')

### Gathering Data 

In [ ]:
base_path = 'Dataset'
df_interaction = pd.read_csv(base_path + "/server_interaction.csv")
df_server = pd.read_csv(base_path + "/server_data.csv")
df_user = pd.read_csv(base_path + "/user_data.csv")
df_category = pd.read_csv(base_path + "/server_category.csv")

df_interaction.head()

,user_id,user_name,server_id,server_name,rating,server_category
0,user_01c9fb9caa27,Sophia Brown,9550bcac-6844-472f-b142-8bff64f42009,Art Lounge Connect,1.0,"[3, 12]"
1,user_135b81fdbcd2,John Smith,f08a46ea-b680-46d7-8e6e-ffe744b2497b,Gaming Hub Connect,NaN,[13]
2,user_135b81fdbcd2,John Smith,83d39e6b-ce67-468d-872e-956e277e8b4c,Gaming Hub Circle,2.0,"[4, 14, 3]"
3,user_135b81fdbcd2,John Smith,62c7b792-6424-4f36-adba-f1fe8e4e96bc,Study Zone Circle,2.0,[7]
4,user_135b81fdbcd2,John Smith,8bf6e9ec-50d8-4e17-af1f-516deb6c7ae1,Art Lounge Circle,4.0,"[7, 4, 13]"


In [33]:
df_server.head()

,server_id,server_name,server_category,server_num
0,0007f352-7249-45f2-a8fa-3e9e68211472,Finance World Corner,"[1, 12, 13]",0
1,002468de-aae1-42a0-8e61-1609d8df4269,Finance World Corner,"[8, 12]",1
2,014f84ea-afd2-482a-974a-51a2aa81c1c2,Art Lounge Corner,[11],2
3,01ffb523-c74e-430d-bd9d-8f794f2d1eaf,Tech Talks Fans,[4],3
4,0259a8db-4a52-492a-a0c8-fed4945fd2c8,Art Lounge Fans,"[2, 11]",4


In [34]:
df_user.head()

,user_id,user_name,interest,user_num
0,user_000079613cbf,Emma Wilson,"[3, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15]",0
1,user_0000b1756c2b,Noah Taylor,"[3, 4, 5, 7, 9, 13]",1
2,user_000138baf092,John Wilson,"[3, 4, 6, 8, 9, 10, 11, 13, 14]",2
3,user_0001e82b5908,Noah Taylor,"[2, 3, 6, 8, 9, 11, 12, 13, 15]",3
4,user_0002454e2a97,Olivia Brown,"[1, 4, 7, 8, 10, 11, 12, 13, 15]",4


In [35]:
df_category.head()

print("Server Interactions: \nRows: ", df_interaction.shape[0], " | Columns: ", df_interaction.shape[1])
print("Server Records: \nRows: ", df_server.shape[0], " | Columns: ", df_server.shape[1])
print("User Records: \nRows: ", df_user.shape[0], " | Columns: ", df_user.shape[1])
print("Server Categories: \nRows: ", df_category.shape[0], " | Columns: ", df_category.shape[1])

Server Interactions: 
Rows:  110625  | Columns:  6
Server Records: 
Rows:  1000  | Columns:  4
User Records: 
Rows:  20000  | Columns:  4
Server Categories: 
Rows:  15  | Columns:  2


### Defining Functions

### Data Preprocessing

In [37]:
category_mapping = set(df_category['Category ID'])

In [38]:
user_map = df_user.set_index('user_id')['user_num']
server_map = df_server.set_index('server_id')['server_num']
df_interaction['user_num'] = df_interaction['user_id'].map(user_map)
df_interaction['server_num'] = df_interaction['server_id'].map(server_map)

In [39]:
df_interaction['rating'] = df_interaction['rating'].fillna(0).astype(float)

In [40]:
df_user['interest'] = df_user['interest'].apply(eval)
df_server['server_category'] = df_server['server_category'].apply(eval)

In [41]:
server_features = [
    (row['server_num'], row['server_category']) for _, row in df_server.iterrows()
]
user_features = [
    (row['user_num'], row['interest']) for _, row in df_user.iterrows()
]

### Preparing Data For Model

In [47]:
dataset = Dataset()
dataset.fit(
    users=df_user['user_num'].unique(),
    items=df_server['server_num'].unique(),
    user_features=category_mapping,
    item_features=category_mapping,
)

In [48]:
interactions, weights = dataset.build_interactions([
    (row['user_num'], row['server_num'], row['rating']) for _, row in df_interaction.iterrows()
])

In [49]:
user_features_matrix = dataset.build_user_features(user_features)
server_features_matrix = dataset.build_item_features(server_features)

### Training Model

In [51]:
model = LightFM(no_components=150, learning_rate=0.05, loss='warp', random_state=42)

In [52]:
model.fit(interactions, item_features=server_features_matrix,user_features=user_features_matrix, sample_weight=weights, epochs=30, num_threads=4)

In [ ]:
model_save_path = "Model/lightfm_model.pkl"

with open(model_save_path, 'wb') as f:
    pickle.dump(model, f)

print(f"Model saved to {model_save_path}")

Model saved to lightfm_model.pkl


### Evaluating Model

In [53]:
precision = precision_at_k(model, interactions, k=5, 
                           user_features=user_features_matrix, 
                           item_features=server_features_matrix).mean()
recall = recall_at_k(model, interactions, k=5, 
                     user_features=user_features_matrix, 
                     item_features=server_features_matrix).mean()
auc = auc_score(model, interactions, 
                user_features=user_features_matrix, 
                item_features=server_features_matrix).mean()

print(f'Precision@k: {precision:.4f}')
print(f'Recall@k: {recall:.4f}')
print(f'AUC: {auc:.4f}')

Precision@k: 0.0689
Recall@k: 0.1445
AUC: 0.9117


### Recommend Servers

In [57]:
user_num = 5  

all_server_scores = model.predict(
    user_ids=user_num, 
    item_ids=np.arange(len(df_server)),
    user_features=user_features_matrix, 
    item_features=server_features_matrix
)

top_servers = np.argsort(-all_server_scores)[:10] 
recommended_servers = df_server.loc[df_server['server_num'].isin(top_servers), 'server_name']

print(f"Top recommended servers for user {user_num}:")
recommended_servers.tolist()

Top recommended servers for user 5:


['Tech Talks Fans',
 'Gaming Hub Fans',
 'Study Zone Circle',
 'Gaming Hub Circle',
 'Study Zone Circle',
 'Tech Talks Circle',
 'Tech Talks Connect',
 'Finance World Connect',
 'Study Zone Fans',
 'Tech Talks Circle']